This is transfer learning CNN codes; VGG16 is used for transfer learning

Mount GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Install the necessary packages or libraries

In [ ]:
import tensorflow as tf 
import numpy as np
import keras
import os
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import matplotlib as mlp
import matplotlib.pyplot as plt
from matplotlib.image import imread
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from PIL import Image

load VGG16 structure (delete fully connected layer)

In [ ]:
model_vgg = VGG16(weights='imagenet', include_top=False)

Define Path, folder and dirname in this step, to be used later

In [ ]:
# define path, folder and dirname(pokemon name folder)
path = '/content/gdrive/My Drive/dlpokedex/dataset'
print(path) # optional, show the path as printed

define a method to load and preprocess the image

In [ ]:
def modelProcess(img_path, model):
    img = load_img(img_path, target_size=(224, 224)) # read image path, resize to 224,224, required by VGG16
    img = img_to_array(img) # transfer to Image Array
    x = np.expand_dims(img, axis=0) # add one dimention to load into VGG
    x = preprocess_input(x) # preprocessing
    x_vgg = model.predict(x) #otbain features，the shape before Fully Connected layer

    x_vgg = x_vgg.reshape(1, 25088) # Connect to FC layer
    return x_vgg

list file names of the training datasets: Path=...dlpokedex; folders=pokemon folders

In [ ]:
# list file names of the training datasets
def transform_format(path):  # transform format
    folders = os.listdir(path)  # read pokemon labels(folders), so we only need specify path in the codes
    for j in range(len(folders)):
        dirName = path + '//' + folders[j] + '//'  # folders have name of pokemon
        li = os.listdir(dirName)  # all pokemon images are named after its folder name, dirname=path/folder
        for filename in li:
            newname = filename
            newname = newname.split(".")  # filename is each pokemon picture and divided by "."
            if newname[-1] != "png":  # reformate all images to png format
                newname[-1] = "png"
                newname = str.join(".", newname)  # use str.join here
                filename = dirName + filename
                newname = dirName + newname
                os.rename(filename, newname)  # rename to newname
                print('reading the images:%s' % (newname))  # optional step to see if there is bugs in reading images
                a = np.array(Image.open(newname))  # read image array
                if ((len(a.shape) != 3) or (a.shape[2] != 3)):  # check if all images are in RGB format
                    a = np.array(Image.open(newname).convert('RGB'))  # change to RGB
                    img = Image.fromarray(a.astype('uint8'))  # form image
                    img.save(newname)  # replace old image
                    print(a.shape)  # test print
    print("all images transfered to PNG file")
    print("all images transfered to RGB format")

Start processing

In [ ]:
def read_data(path):
    folders = os.listdir(path) # read image folders, for example 5 folders if 5 pockemon
    for j in range(len(folders)): # cycle # = folder #
        folder = path + '//' + folders[j] # folders, or pockemon labels
        dirs = os.listdir(folder) # read all images under each folder
        # path for image
        img_path = []
        for i in dirs:
            if os.path.splitext(i)[1] == ".png": # has done transformation into png image
                img_path.append(i)
        img_path = [folder + "//" + i for i in img_path] 
        # full path of each image
        
        # start processing
        features1 = np.zeros([len(img_path), 25088]) # array each image
        for i in range(len(img_path)):
            feature_i = modelProcess(img_path[i], model_vgg) # process image by image
            print('preprocessed:', img_path[i])
            features1[i] = feature_i
        if j == 0: # add all images array together
            X = features1 # first cycle
        else:
            X = np.concatenate((X, features1), axis=0)
            # concatenate image after image
    return X # all images are concatenated together at last

read the images data and label images with 5 pokemon names(folder name), takes long time to process

In [ ]:
path = '/content/gdrive/My Drive/dlpokedex/dataset' # Path to dataset
# one-hot encoding
y = []
folders = os.listdir(path)  # load path of dataset
for j in range(len(folders)): # j: 0, 1, 2, 3, 4 ; 
    dirName = path + '//' + folders[j] + '//'  # folder path
    lens = len(os.listdir(dirName)) # number of images under each Pokemon label (folder)
    for i in range(lens):
        y.append(j) # append images

lb = LabelBinarizer()
y = lb.fit_transform(y) # one-hot encoding

transform_format(path)  # format transformation
X = read_data(path) # read data
print('X.shape:', X.shape)
print('y.shape:', y.shape)
print('-' * 35)


## Above output(now shown) show the processing of transfer images to png file, in RGB format, resize to (244x244), label of images, i.e. the process defined in previous steps

In [ ]:
# partition train and test(validation) data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
print('X_train.shape:', X_train.shape)
print('X_test.shape:', X_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)


Start model

In [ ]:
# initialize the model
model = Sequential()
model.add(Dense(units=40, activation='relu', input_dim=25088))

model.add(Dense(units=5, activation='softmax'))
# if label of Pokemon =5 ，then units=5，multicategory we use softmax，if binary we use sigmoid
model.summary()
# check model structure


model started successfully 

Fit the model

In [ ]:
# configure the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# optimizer Adam，define loss function，here we use categorical_crossentropy，Metrics is accuracy
# train the model, modified for both train and test(validation), add batch size, 
hist=model.fit(X_train, y_train, batch_size=32, epochs=20, verbose=1,
               validation_data =(X_test, y_test))
# start training，epochs=20
# define variables for accuracy and loss ploting
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['accuracy']
val_acc=hist.history['val_accuracy']
xc=range(20) # 20 is the epochs number, modify if epochs number change

Now we see the history of epochs for both train and validation data

Run model for 20 epochs successfully :)

In [ ]:
# summary the model
model.summary()


Plot accuracy and loss

In [ ]:
plt.figure(1, figsize=(7, 5))
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train', 'val'])
# print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.savefig('trial_psi_loss.png', bbox_inches='tight')

plt.figure(2, figsize=(7, 5))
plt.plot(xc, train_acc)
plt.plot(xc, val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train', 'val'], loc=4)
# print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.savefig('trial_psi_acc.png', bbox_inches='tight')


Plot the Confusion Matrix for validation data(X_test, y_test)

In [ ]:
# Train dataset decoding, optional
y_train_predict = model.predict_classes(X_train)  
y_trainnew = lb.inverse_transform(y_train)
# decoding from one-hot encoding


# valication(test) accuracy, y_test_predict and y_testnew will be used to plot confusion matrix
y_test_predict = model.predict_classes(X_test) 
y_testnew = lb.inverse_transform(y_test)
# decoding from one-hot encoding


Define confusion matrix (cm)

In [ ]:
# import libraries and define cm(confusion matrix)
from sklearn import metrics
from sklearn.metrics import confusion_matrix
cm = metrics.confusion_matrix(y_testnew, y_test_predict)

Define a function to plot the confusion matrix

In [ ]:
#Defining function for confusion matrix plot
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Computing confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

# Visualizing
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

   # Rotating the tick labels and setting their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Looping over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
np.set_printoptions(precision=2)

Specify a Pokemon index to display in confusion matrix, i.e. class_names

In [ ]:
class_names= folders
# folders are the index of Pokemon folders defined in earlier steps of the model building

Plot non-nomalized cm

In [ ]:
#Non-Normalized Confusion Matrix
plt.figure(figsize=(20,20))
plot_confusion_matrix(y_testnew, y_test_predict, classes = class_names, title='Non-Normalized Confusion matrix')
plt.show()

Plot normalized cm

In [ ]:
#Normalized Confusion Matrix
plt.figure(figsize=(35,35))
plot_confusion_matrix(y_testnew, y_test_predict, classes = class_names, normalize=True, title='Non-Normalized Confusion matrix')
plt.show()

Save the model

In [ ]:
model.save('/content/gdrive/My Drive/dlpokedex/Pokedex_modelTL.h5')  # save your model with h5py